<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [ ]:
%pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1 numpy==1.23.5 mlflow jupyter xgboost

In [ ]:
import numpy as np
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [6]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [7]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [10]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [21]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": "liblinear"}, 
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [22]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [ ]:
# dagshub setup

# import dagshub
# dagshub.init(repo_owners='', repo_name='', mlflow=True)

In [ ]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = '',
os.environ['MLFLOW_TRACKING_PASSWORD'] = '',
os.environ['MLFOW_TRACKING_URI'] = 'https://dagshub.com/...'



# Initialize MLflow
mlflow.set_experiment("Anomaly-Detection")
mlflow.set_tracking_uri("http://localhost:5000")

mlflow.set_tracking_uri("https://dagshub.com/...")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        
        # Log all model hyperparameters
        mlflow.log_params(params)
        mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_class_1': report['1']['recall'],
        'recall_class_0': report['0']['recall'],
        'f1_score_macro': report['macro avg']['f1-score']   
    })
 
        
        # Log model artifact
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")


2025/06/29 04:50:44 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly-Detection' does not exist. Creating a new experiment.
2025/06/29 04:50:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/29 04:50:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/29 04:50:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/610993840370170556/runs/481ae567ff9e4e20828bc4d8ed327a08
🧪 View experiment at: http://localhost:5000/#/experiments/610993840370170556


2025/06/29 04:50:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/29 04:50:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/610993840370170556/runs/913a7dc85a6a46f995a7cd8604fdbb79
🧪 View experiment at: http://localhost:5000/#/experiments/610993840370170556


2025/06/29 04:51:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/29 04:51:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/610993840370170556/runs/3d3c97f8fe5848768c29a5510485cf80
🧪 View experiment at: http://localhost:5000/#/experiments/610993840370170556


2025/06/29 04:51:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/610993840370170556/runs/598752a3dab54f988d897b850ad5baa2
🧪 View experiment at: http://localhost:5000/#/experiments/610993840370170556


Register Model

In [27]:
model_name = "XGB-Smote"
run_id=input("Enter Run ID:")
model_uri= f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)

Successfully registered model 'XGB-Smote'.
2025/06/29 05:05:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 598752a3dab54f988d897b850ad5baa2 has no artifacts at artifact path 'model', registering model based on models:/m-8d78bc1a58814f0292a19c961846acd4 instead
2025/06/29 05:05:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


Load the Model

In [ ]:
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

#model_url = f"models:/{model_name}@alias"
#model_uri = f"models:/{model_name}/@top"
# in this case version number is not required since avery model has unique alias

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [31]:
model_version = 1
dev_model_uri = f"models:/{model_name}/{model_version}"
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1751156153945, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1751156153945, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='598752a3dab54f988d897b850ad5baa2', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>